<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
### YOUR CODE STARTS HERE

#import pandas
#import stats from scipi
#import numpy

import pandas as pd
from scipy import stats
import numpy as np

In [0]:
# read in data set

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                 names= ['Class Name', 'handicapped-infants', 'water-project-cost-sharing',
                         'adopotion-of-the-budget-resolution', 'physician-fee-freeze',
                         'el-salvaldor-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 
                         'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 
                         'synfuels-corporation-cutback', 'education-spending',
                         'superfund-right-to-sue', 'crime', 'duty-free-exports',
                         'export-administration-act-south-africa'])

print(df.shape)
df.head()

(435, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adopotion-of-the-budget-resolution,physician-fee-freeze,el-salvaldor-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# update 'y' == 1, update 'n' == 0, update '?' == NaN

df = df.replace({'y': 1, 'n': 0, "?": np.NaN})
print(df.shape)
df.head()

(435, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adopotion-of-the-budget-resolution,physician-fee-freeze,el-salvaldor-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
#filter data set into democrate and republican

rep = df[df['Class Name']=='republican']

print(rep.shape)
rep.head()

(168, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adopotion-of-the-budget-resolution,physician-fee-freeze,el-salvaldor-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
dem = df[df['Class Name']=='democrat']

print(dem.shape)
dem.head()


(267, 17)


,Class Name,handicapped-infants,water-project-cost-sharing,adopotion-of-the-budget-resolution,physician-fee-freeze,el-salvaldor-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
# T-Test for handicapped-infants
print(rep['handicapped-infants'].mean())
print(dem['handicapped-infants'].mean())
stats.ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit')

0.18787878787878787
0.6046511627906976


Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [0]:




def t_test(columns):
    print( columns )
    print('the mean for republicans is: ')
    print(rep[columns].mean())

    print('the mean for democrats is: ')
    print(dem[columns].mean())

    return stats.ttest_ind(rep[columns], dem[columns], nan_policy='omit')

In [0]:
# Test for handicapped infants
t_test('handicapped-infants')

handicapped-infants
the mean for republicans is: 
0.18787878787878787
the mean for democrats is: 
0.6046511627906976


Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [0]:
# Test for water-project-cost-sharing
t_test('water-project-cost-sharing')

water-project-cost-sharing
the mean for republicans is: 
0.5067567567567568
the mean for democrats is: 
0.502092050209205


Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
# Test for adopotion-of-the-budget-resolution
t_test('adopotion-of-the-budget-resolution')

adopotion-of-the-budget-resolution
the mean for republicans is: 
0.13414634146341464
the mean for democrats is: 
0.8884615384615384


Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
# Test for physician-fee-freeze	
t_test('physician-fee-freeze')

physician-fee-freeze
the mean for republicans is: 
0.9878787878787879
the mean for democrats is: 
0.05405405405405406


Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
# Test for el-salvaldor-aid		
t_test('el-salvaldor-aid')

el-salvaldor-aid
the mean for republicans is: 
0.9515151515151515
the mean for democrats is: 
0.21568627450980393


Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

In [0]:
# Test for religious-groups-in-schools			
t_test('religious-groups-in-schools')

religious-groups-in-schools
the mean for republicans is: 
0.8975903614457831
the mean for democrats is: 
0.47674418604651164


Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

In [0]:
# Test for anti-satellite-test-ban			
t_test('anti-satellite-test-ban')

anti-satellite-test-ban
the mean for republicans is: 
0.24074074074074073
the mean for democrats is: 
0.7722007722007722


Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)

In [0]:
# Test for aid-to-nicaraguan-contras			
t_test('aid-to-nicaraguan-contras')

aid-to-nicaraguan-contras
the mean for republicans is: 
0.15286624203821655
the mean for democrats is: 
0.8288973384030418


Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

In [0]:
# Test for mx-missile				
t_test('mx-missile')

mx-missile
the mean for republicans is: 
0.11515151515151516
the mean for democrats is: 
0.7580645161290323


Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)

In [0]:
# Test for immigration				
t_test('immigration')

immigration
the mean for republicans is: 
0.5575757575757576
the mean for democrats is: 
0.4714828897338403


Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [0]:
# Test for synfuels-corporation-cutback					
t_test('synfuels-corporation-cutback')

synfuels-corporation-cutback
the mean for republicans is: 
0.1320754716981132
the mean for democrats is: 
0.5058823529411764


Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)

In [0]:
# Test for education-spending					
t_test('education-spending')

education-spending
the mean for republicans is: 
0.8709677419354839
the mean for democrats is: 
0.14457831325301204


Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

In [0]:
# Test for superfund-right-to-sue					
t_test('superfund-right-to-sue')

superfund-right-to-sue
the mean for republicans is: 
0.8607594936708861
the mean for democrats is: 
0.2896825396825397


Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)

In [0]:
# Test for crime					
t_test('crime')

crime
the mean for republicans is: 
0.9813664596273292
the mean for democrats is: 
0.35019455252918286


Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

In [0]:
# Test for duty-free-exports					
t_test('duty-free-exports')

duty-free-exports
the mean for republicans is: 
0.08974358974358974
the mean for democrats is: 
0.6374501992031872


Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32)

In [0]:
# Test for export-administration-act-south-africa					
t_test('export-administration-act-south-africa')

export-administration-act-south-africa
the mean for republicans is: 
0.6575342465753424
the mean for democrats is: 
0.9351351351351351


Ttest_indResult(statistic=-6.849454815841208, pvalue=3.652674361672226e-11)

For the above 2 Sample T-Test assuming a $H_0: mean for republican == mean for democrat. At a P-value 0f .01 (confidence interval of 99%), we see that the we would **FAIL TO REJECT** the Null hypothesis for the below categories of votes:

1)water-project-cost-sharing

2)immigration



Democrats support the following (on a statistically significant basis) more than republicans:

1)handicapped infants

2)adopotion-of-the-budget-resolution

3)anti-satellite-test-ban

4)aid-to-nicaraguan-contras

5)synfuels-corporation-cutback

6)mx-missile

7)duty-free-exports

8)export-administration-act-south-africa

Republicans support the following (on a statistically significant basis) more than Democrats:

1)physician-fee-freeze

2)el-salvaldor-aid

3)religious-groups-in-schools

4)education-spending

5)superfund-right-to-sue

6)crime

In [0]:
issues = list(df.loc[:, df.columns != 'Class Name'])
issues


['handicapped-infants',
 'water-project-cost-sharing',
 'adopotion-of-the-budget-resolution',
 'physician-fee-freeze',
 'el-salvaldor-aid',
 'religious-groups-in-schools',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'immigration',
 'synfuels-corporation-cutback',
 'education-spending',
 'superfund-right-to-sue',
 'crime',
 'duty-free-exports',
 'export-administration-act-south-africa']

In [0]:
### Function to print all t-test statistics and means in one code run

# def t_test_final(df, df1):

#   issues = list(df.loc[:, df.columns != 'Class Name'])

#   for issue in issues:
#     print(issue)
#     print('\n')
#     print('Mean of Parties: ')
#     print('Republican Mean is: ' + str(rep[issue].mean()))
#     print('Democratic Mean is: ' + str(dem[issue].mean()))
#     print('___'*30)
#     print(stats.ttest_ind(df[issue], df1[issue], nan_policy='omit'))
#     print('\n')
#     print('~~~'*30)
 

def t_test_final(df, df1):

  issues = list(df.loc[:, df.columns != 'Class Name'])

  for issue in issues:
    print(issue)
    print('\n')
    print('Mean of Parties: ')
    print('Republican Mean is: ' + str(rep[issue].mean()))
    print('Democratic Mean is: ' + str(dem[issue].mean()))
    print('___'*13)
    print('\n')
    T_Statistic, P_value = stats.ttest_ind(df[issue], df1[issue], nan_policy='omit')
    print('T_Statistic is: ' + str(T_Statistic))
    print('P_value is: ' + str(P_value))
    print('\n')
    print('~~~'*30)
  

In [160]:
t_test_final(rep, dem)

handicapped-infants


Mean of Parties: 
Republican Mean is: 0.18787878787878787
Democratic Mean is: 0.6046511627906976
_______________________________________


T_Statistic is: -9.205264294809222
P_value is: 1.613440327937243e-18


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
water-project-cost-sharing


Mean of Parties: 
Republican Mean is: 0.5067567567567568
Democratic Mean is: 0.502092050209205
_______________________________________


T_Statistic is: 0.08896538137868286
P_value is: 0.9291556823993485


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
adopotion-of-the-budget-resolution


Mean of Parties: 
Republican Mean is: 0.13414634146341464
Democratic Mean is: 0.8884615384615384
_______________________________________


T_Statistic is: -23.21277691701378
P_value is: 2.0703402795404463e-77


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
physician-fe

In [0]:
t_value, p_value = stats.ttest_ind(rep['adopotion-of-the-budget-resolution'], dem['adopotion-of-the-budget-resolution'], nan_policy='omit')

print(t_value)
print(p_value)

-23.21277691701378
2.0703402795404463e-77


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!